In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as sg
import xlrd
import os

In [2]:
#1/3倍频程中心频率
freqs_3 = np.array([10e+01, 
                    1.25e+01, 1.60e+01, 2.00e+01, 2.50e+01, 3.15e+01, 4.00e+01, 
                    5.00e+01, 6.30e+01, 8.00e+01, 1.00e+02, 1.25e+02, 1.60e+02, 
                    2.00e+02, 2.50e+02, 3.15e+02, 4.00e+02, 5.00e+02, 6.30e+02, 
                    8.00e+02, 1.00e+03, 1.25e+03, 1.60e+03, 2.00e+03, 2.50e+03,
                    3.15e+03, 4.00e+03, 5.00e+03, 6.30e+03, 8.00e+03, 1.00e+04, 
                    1.25e+04, 1.60e+04,2.00e+04])
#p0
p0 = 2.00e-5
print(freqs_3 [3:33])

[   20.     25.     31.5    40.     50.     63.     80.    100.    125.
   160.    200.    250.    315.    400.    500.    630.    800.   1000.
  1250.   1600.   2000.   2500.   3150.   4000.   5000.   6300.   8000.
 10000.  12500.  16000. ]


In [3]:
#A计权系数
Wght_fct = np.array([-70.4,
                     -63.4, -56.7, -50.7, -44.7, -39.4, -34.6, 
                     -30.2, -26.2, -22.5, -19.1, -16.1, -13.4, 
                     -10.9, -8.6, -6.6, -4.8,  -3.2, - 1.9,
                     -0.8, 0.0, 0.6, 1.0, 1.2, 1.3,
                     1.2, 1.0, 0.5, -0.1, -1.1, -2.5,
                     -4.3, -6.6, -9.3])
#print(Wght_fct [3:33])

In [4]:
def nextpow2(max_ele):
    n = 0
    while 2**n < max_ele:
        n += 1
    return 2**n

In [5]:
def getAllImages(path):
    #f.endswith（）  限制文件类型
    #f.endswith('.jpg')|f.endswith('.png')  改成这句可以读取jpg/png格式的文件
    #注意 返回的是绝对路径
   return [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.xlsx')]

In [6]:
def calculate(address, measure_frqs, freqs_order, Weighting_factor,channal):
    
    #读取数据
    workbook = xlrd.open_workbook(address)
    Data_sheet = workbook.sheets()[0] 
    #声压时程
    t = Data_sheet.col_values(0)
    p_t  = Data_sheet.col_values(channal)
    data_len = len(p_t)
    #plt.figure(figsize=(14,8), dpi=200)
    #plt.plot(t,p_t)
    
    #加汉宁窗(恢复系数为1.633)
    #p_t = 1.633 * np.hamming(data_len) * p_t
    p_t = 1.633 * np.hanning(data_len) * p_t
    nfft = nextpow2(data_len)
    p_f = np.fft.fft(p_t,nfft)
    fx =  measure_frqs/2 * np.linspace(0,1,nfft/2)
    w = 2 * np.abs(p_f[:int(nfft/2)]/data_len)
    #w = np.abs(np.abs(p_f[:int(nfft/2)]/data_len))
    #绘制频谱
    #plt.figure(figsize=(14,8), dpi=200)
    #plt.semilogx(fx,w)
    
    
    
     # 求取1/3频程起止频率
    f_start =  freqs_order * 2 ** (-1 / 6)  
    f_stop = freqs_order * 2 ** (1 / 6)  
    
    #print(f_start)
    #分频带滤波   
    
    m = len(freqs_order)-1
    y_out = np.zeros(m)
    #'''
    #print(index_start,index_end)
    for i in range(m):
        
        index_start = int(np.round(f_start[i]*nfft/measure_frqs))
        index_end = int(np.round(f_start[i+1]*nfft/measure_frqs))
        index_end = int(np.round(f_stop[i]*nfft/measure_frqs))
        #index_end = int(np.round(f_start[i+1]*nfft/measure_frqs))
        data = np.zeros(nfft,dtype=np.complex)
        data[index_start:index_end] = p_f[index_start:index_end]
        data[nfft-1-index_end:nfft-1-index_start] = p_f[nfft-1-index_end:nfft-1-index_start]
        if f_stop[i]>measure_frqs/2:
            y_out[i] = p0
        else:
            data_ifft = np.fft.ifft(data,nfft)
            y_out[i] = np.power(np.var(data_ifft[:data_len].real),0.5)
    
    #print(y_out)
    '''
    #for i in range(m):   
        index = np.where((fx >= f_start[i]) & (fx < f_stop[i]) & ( f_stop[i]<measure_frqs/2))  # 取交集
        data = np.zeros(int(nfft/2))
        
        data[index] = np.take(p_f, index)[0]
        u=data[::-1]
        data_a = np.append(data,u)
        
        if data_a.any() == 0:
            y_out[i] = p0
        else:
            data_ifft = np.fft.ifft(data_a,nfft)
            y_out[i] = np.power(np.var(data_ifft.real[:data_len]),0.5)
    print(y_out)
    '''
    y_out_1 = 10*np.log10(y_out**2/p0**2)
    y_out_A = 10*np.log10(np.power(10,(y_out_1+Wght_fct[3:33])/10))
    non_out = 10*np.log10(sum(np.power(10,y_out_1/10)))
    A_out = 10*np.log10(sum(np.power(10,(y_out_1+Wght_fct[3:33])/10)))
    
    return y_out_1, y_out_A,non_out,A_out 

In [12]:
if __name__ == '__main__':
    
    measure_frqs = 20000
    
    for path in getAllImages(r"C:/Users/44581/Desktop/test/7.31"):
        print(path,"\n")
        
    #读入文件地址
    #ads = 'C:/Users/44581/Desktop/test/7.31_6-8_9.xlsx'
    #采集频率
        for i in range(13,16):
            out_non,out_A, non, A = calculate(path, measure_frqs, freqs_3[3:34], Wght_fct[3:33],i)
            print("number of chanal: " , i)
            print("non=",non)
            print(out_non,"\n")
            print("A=",A)
            print(out_A,"\n")
            #plt.figure(figsize=(14,8), dpi=80)
            #plt.bar(np.linspace(0,30,30),out_A,1,align='center')

C:/Users/44581/Desktop/test/7.31\7.31_6-8_1.xlsx 



D:\anaconda\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


number of chanal:  13
non= 98.37483987566893
[75.6550963  79.5334981  74.48415449 74.1849701  83.01061604 77.11324394
 79.82074938 81.75034605 92.07835569 85.76424836 83.09699365 89.12413274
 89.29533549 84.36760682 87.18625277 87.71335777 86.91737275 83.23919351
 78.50372921 73.86100296 76.29496304 77.22841821 75.55007561 69.68929128
 68.49732829 65.40107938 62.71638614  0.          0.          0.        ] 

A= 93.07135872135768
[24.9550963  34.8334981  35.08415449 39.5849701  52.81061604 50.91324394
 57.32074938 62.65034605 75.97835569 72.36424836 72.19699365 80.52413274
 82.69533549 79.56760682 83.98625277 85.81335777 86.11737275 83.23919351
 79.10372921 74.86100296 77.49496304 78.52841821 76.75007561 70.68929128
 68.99732829 65.30107938 61.61638614 -2.5        -4.3        -6.6       ] 

number of chanal:  14
non= 97.24182052978531
[81.64144024 82.37889406 79.48669897 78.58127    83.66376699 77.77322196
 79.01052731 80.63514701 91.15518088 84.25925873 81.98564873 86.79016835
 87.468

number of chanal:  15
non= 97.23396222652896
[76.23361896 83.76429497 75.32236682 75.92728324 85.27130077 75.56220278
 76.91996687 78.66529644 91.00846925 84.28271855 81.51343384 86.68137945
 89.27478473 83.22205603 85.9332677  83.5392784  82.58892571 81.93620672
 79.90745414 74.9864365  77.32390185 77.93850077 75.15441901 69.14733414
 65.43811447 61.36142755 57.68848446  0.          0.          0.        ] 

A= 91.40631645203194
[25.53361896 39.06429497 35.92236682 41.32728324 55.07130077 49.36220278
 54.41996687 59.56529644 74.90846925 70.88271855 70.61343384 78.08137945
 82.67478473 78.42205603 82.7332677  81.6392784  81.78892571 81.93620672
 80.50745414 75.9864365  78.52390185 79.23850077 76.35441901 70.14733414
 65.93811447 61.26142755 56.58848446 -2.5        -4.3        -6.6       ] 

C:/Users/44581/Desktop/test/7.31\7.31_6-8_4.xlsx 

number of chanal:  13
non= 99.0111767292895
[75.65605464 78.99879241 74.99526393 74.02684608 89.2307538  73.939911
 77.14253672 79.03660083 92.1162

number of chanal:  13
non= 98.09292840234625
[74.21601753 76.85581609 73.67577219 73.34351639 85.26087996 73.89698785
 76.90950013 79.55493476 90.75433085 85.72871313 83.18323542 88.92776771
 89.23245547 84.69823166 87.48930152 87.56348648 86.66237571 84.00674237
 78.81831961 74.51049999 77.48000916 78.95889851 75.65413969 69.49963639
 67.50046281 63.74364844 59.58500312  0.          0.          0.        ] 

A= 93.22222024089528
[23.51601753 32.15581609 34.27577219 38.74351639 55.06087996 47.69698785
 54.40950013 60.45493476 74.65433085 72.32871313 72.28323542 80.32776771
 82.63245547 79.89823166 84.28930152 85.66348648 85.86237571 84.00674237
 79.41831961 75.51049999 78.68000916 80.25889851 76.85413969 70.49963639
 68.00046281 63.64364844 58.48500312 -2.5        -4.3        -6.6       ] 

number of chanal:  14
non= 96.65943752929513
[79.49998813 82.09758351 77.41255708 75.40007874 82.99870777 74.33019415
 75.99297444 79.14846373 89.53527625 83.85700591 81.95069669 87.43000175
 86.885